# CIS468 Project Report - Auto Tagger Todo-List DB Version 2

project/

todos:



## Initialize the Envrionment

In [ ]:
# install the spaCy model
!pip install -U spacy
!python -m spacy download en_core_web_lg
!pip install sqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


## Corpus

In [ ]:
# Not Used, Generated by ChatGPT.

corpus = [
    # ===== Sports =====
    ("Play basketball with Alex at Central Park tomorrow 3pm", "sports"),
    ("Join soccer practice at 5:30pm this Friday", "sports"),
    ("Morning run around the lake at 6am", "sports"),
    ("Tennis match with Sarah next Saturday afternoon", "sports"),
    ("Sign up for yoga class this evening", "sports"),
    ("Watch Lakers game with Mike tonight at 8", "sports"),
    ("Buy new running shoes before next week's marathon", "sports"),
    ("Hiking trip with family this weekend", "sports"),
    ("Swim practice at community center tomorrow morning", "sports"),
    ("Organize basketball tournament for team next month", "sports"),

    # ===== Cooking =====
    ("Cook dinner for family tonight", "cooking"),
    ("Try new pasta recipe tomorrow evening", "cooking"),
    ("Bake birthday cake for mom this Saturday", "cooking"),
    ("Prepare lunch for office potluck next Friday", "cooking"),
    ("Make vegetarian lasagna for dinner party", "cooking"),
    ("Experiment with sushi making this weekend", "cooking"),
    ("Grill steaks for backyard BBQ on Sunday", "cooking"),
    ("Learn to make French toast tomorrow morning", "cooking"),
    ("Cook Thanksgiving turkey next Thursday", "cooking"),
    ("Prepare meal prep for next week on Sunday", "cooking"),

    # ===== Gaming =====
    ("Play Call of Duty with friends tonight at 9pm", "gaming"),
    ("Join Minecraft server this weekend", "gaming"),
    ("Finish Zelda: Tears of the Kingdom by next week", "gaming"),
    ("Online chess tournament Saturday afternoon", "gaming"),
    ("Stream Fortnite gameplay tomorrow evening", "gaming"),
    ("Buy new gaming headset before next month", "gaming"),
    ("LAN party at Mark's place this Friday", "gaming"),
    ("Practice for esports competition next month", "gaming"),
    ("Update gaming PC components this weekend", "gaming"),
    ("Try new VR game with Alex tomorrow", "gaming"),

    # ===== Work =====
    ("Finish quarterly report by next Friday", "work"),
    ("Prepare presentation for Monday meeting", "work"),
    ("Conference call with Tokyo team at 9am tomorrow", "work"),
    ("Submit expense reports before end of month", "work"),
    ("Interview candidates for developer position", "work"),
    ("Review project timeline with team this afternoon", "work"),
    ("Attend leadership workshop next Wednesday", "work"),
    ("Complete budget planning by end of week", "work"),
    ("Organize team building event next month", "work"),
    ("Update company website content tomorrow", "work"),

    # ===== Study =====
    ("Study for calculus exam next Monday", "study"),
    ("Complete CIS468 project by next Friday", "study"),
    ("Research paper on machine learning due next month", "study"),
    ("Group study session at library tomorrow 2pm", "study"),
    ("Prepare for TOEFL test this weekend", "study"),
    ("Watch lecture videos on quantum computing", "study"),
    ("Read chapters 5-7 for literature class", "study"),
    ("Practice Python coding exercises tonight", "study"),
    ("Attend machine learning workshop next week", "study"),
    ("Review Spanish vocabulary before class tomorrow", "study"),

    # ===== Shopping =====
    ("Buy groceries at Whole Foods tomorrow", "shopping"),
    ("Get birthday gift for mom this weekend", "shopping"),
    ("Purchase new laptop before next semester", "shopping"),
    ("Clothes shopping at mall next Saturday", "shopping"),
    ("Order kitchen supplies online tonight", "shopping"),
    ("Look for new sofa this weekend", "shopping"),
    ("Get ingredients for dinner party tomorrow", "shopping"),
    ("Christmas gift shopping next month", "shopping"),
    ("Buy new phone case before trip", "shopping"),
    ("Grocery delivery for next week", "shopping")
]

## DataBase

In [ ]:
import sqlite3
import json

class DatabaseManager:
    def __init__(self, db_path='/content/drive/MyDrive/todo_records.db'):
        """Initialize database connection"""
        self.db_path = db_path
        self._create_table()

    def _create_table(self):
        """Create database table if not exists"""
        with sqlite3.connect(self.db_path) as conn:
            c = conn.cursor()
            c.execute('''
                CREATE TABLE IF NOT EXISTS todo_records (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    original_text TEXT NOT NULL,
                    creation_time TEXT NOT NULL,
                    candidate_labels TEXT NOT NULL,
                    predicted_tags TEXT NOT NULL,
                    ner_results TEXT NOT NULL
                )
            ''')
            conn.commit()

    def save_record(self, record):
        """Save record to database"""
        with sqlite3.connect(self.db_path) as conn:
            c = conn.cursor()
            c.execute('''
                INSERT INTO todo_records (
                    original_text,
                    creation_time,
                    candidate_labels,
                    predicted_tags,
                    ner_results
                ) VALUES (?, ?, ?, ?, ?)
            ''', (
                record["original_text"],
                record["creation_time"],
                json.dumps(record["candidate_labels"]),
                json.dumps(record["predicted_tags"]),
                json.dumps(record["ner_results"])
            ))
            conn.commit()
            return c.lastrowid

    def load_records(self):
        """Load all records from database"""
        with sqlite3.connect(self.db_path) as conn:
            c = conn.cursor()
            c.execute("SELECT * FROM todo_records")
            records = []
            for row in c.fetchall():
                records.append({
                    "id": row[0],
                    "original_text": row[1],
                    "creation_time": row[2],
                    "candidate_labels": json.loads(row[3]),
                    "predicted_tags": json.loads(row[4]),
                    "ner_results": json.loads(row[5])
                })
            return records


## NER Recognition

In [16]:
# Initialize the Environment
!pip install dateparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 9.4 MB/s eta 0:00:00


### ner_extractor.py

In [27]:
# ner_extractor.py

'''
todo: complete todos in _extract_and_parse_time
'''

import datetime
import spacy
import dateparser

class EnhancedNERExtractor:
    def __init__(self, model_path="en_core_web_lg"):
        """
        Initialize the NER extractor
        :param model_path: spaCy model path or name
        """
        self.nlp = spacy.load(model_path)
        print(f"Loaded model: {model_path}")

    def extract_entities(self, text, current_time=None, output_format="%Y-%m-%d %H:%M"):
        """
        Enhanced NER extraction
        :param text: Input text
        :param current_time: Current time (datetime object)
        :param output_format: Time output format
        :return: Dictionary containing entity information
        """
        doc = self.nlp(text)

        # Extract basic entities
        entities = {
            "original_text": text,
            "event_time": self._extract_and_parse_time(doc, current_time, output_format),
            "persons": self._extract_persons(doc),

            # Todo: Continue to extract more identities
            # "organizations": self._extract_organizations(doc),
            # "events": self._extract_events(doc),
            # "other_nouns": self._extract_other_nouns(doc)
        }

        return entities


    def _extract_and_parse_time(self, doc, current_time, output_format):
        '''
        todo: add duration field to the modification the time model, to save duration of the event
        todo: Failed to recognize, and consider all of the time expressions
          {'original_text': 'Meet John at Google HQ tomorrow from 3pm to 5pm for product launch', 'event_time': '2023-10-16 14:30', 'persons': ['Meet John'], 'organizations': [], 'events': [], 'other_nouns': ['product', 'launch']}
        todo: Better logic of the time recognition is required
          {'original_text': 'I need to double check TA with my final before the end of today', 'event_time': None, 'persons': [], 'organizations': [], 'events': [], 'other_nouns': ['end', 'today']}
        '''
        time_exprs = []
        # Combine continuous time words (e.g., "next Monday at 3pm")
        for ent in doc.ents:
            if ent.label_ in ["DATE", "TIME"]:
                # Check if the words are adjacent
                if time_exprs and ent.start == time_exprs[-1]["end"]:
                    time_exprs[-1]["text"] += " " + ent.text
                    time_exprs[-1]["end"] = ent.end
                else:
                    time_exprs.append({
                        "text": ent.text,
                        "start": ent.start,
                        "end": ent.end
                    })

        if not time_exprs:
            return None

        # Return the processed time
        for expr in time_exprs:
            parsed_time = dateparser.parse(
                expr["text"],
                settings={'RELATIVE_BASE': current_time}
            )
            if parsed_time:
                return parsed_time.strftime(output_format)

        return None

    def _extract_persons(self, doc):
        """Extract person entities"""
        return [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

    def _extract_organizations(self, doc):
        """Extract organization entities"""
        return [ent.text for ent in doc.ents if ent.label_ in ["ORG", "FAC"]]

    def _extract_events(self, doc):
        """Extract event entities"""
        return [ent.text for ent in doc.ents if ent.label_ == "EVENT"]

    def _extract_other_nouns(self, doc):
        """Extract other important nouns (non-entities)"""
        return [token.text for token in doc if token.pos_ == "NOUN" and not token.ent_type_]

if __name__ == "__main__":
  # --- Sample ---

  # Initialize extractor
  extractor = EnhancedNERExtractor()

  # Current time (example)
  current_time = datetime.datetime.now()

  # Example text
  text = "Meet John at Google HQ tomorrow at 3pm for product launch"
  # text = "Meet John at Google HQ tomorrow from 3pm to 5pm for product launch"
  # text = "I need to double check my final project with Prof.Gabriel before the end of today"

  # Execute extraction
  result = extractor.extract_entities(
      text=text,
      current_time=current_time,
      output_format="%Y-%m-%d %H:%M"
  )
  # Output result
  print(result)

Loaded model: en_core_web_lg
{'original_text': 'Meet John at Google HQ tomorrow at 3pm for product launch', 'event_time': '2025-08-10 03:12', 'persons': ['Meet John']}


### ner_identities_trainer.py

In [17]:
# ner_trainer
'''
todo: train time
todo: train rest of the entities
'''
class NERModelTrainer:
    def __init__(self, base_model="en_core_web_lg"):
        """
        Initialize the model trainer
        :param base_model: Base model name or path
        """
        self.nlp = spacy.load(base_model)
        print(f"Initialized trainer with base model: {base_model}")

    def train_model(self, train_data, output_dir="custom_ner_model", n_iter=20):
        """
        Train a custom NER model
        :param train_data: Training data list [(text, {"entities": [(start, end, label)]})]
        :param output_dir: Model output directory
        :param n_iter: Number of training iterations
        :return: Path to the trained model
        """
        # Add new entity labels (if they don't exist)
        pass

    @staticmethod
    def prepare_training_data(corpus):
        """
        Prepare training data (sample method, needs implementation based on actual corpus)
        :param corpus: Raw corpus
        :return: Formatted training data
        """
        pass
        # return training_data



## Tag Allocation

### text_tager.py

In [ ]:
# Tager

from transformers import pipeline
from typing import List, Dict

class EventTagger:
    def __init__(self, model_name: str = "facebook/bart-large-mnli"):
        """
        Initialize the zero-shot text classification model
        :param model_name: Pretrained model (default: optimal for zero-shot classification)
        """
        self.classifier = pipeline(
            "zero-shot-classification",
            model=model_name,
            device=-1  # -1 for CPU, 0+ for GPU index
        )

    def predict_tags(
        self,
        text: str,
        candidate_labels: List[str],
        multi_label: bool = True,
        threshold: float = 0.5
    ) -> List[str]:
        """
        Predict relevant tags for the event description
        :param text: Event description text
        :param candidate_labels: Set of candidate tags
        :param multi_label: Allow multiple tags (default True)
        :param threshold: Confidence threshold for tags (default 0.5)
        :return: List of predicted tags
        """
        results = self.classifier(
            text,
            candidate_labels,
            multi_label=multi_label
        )
        return [
            label for label, score in zip(results['labels'], results['scores'])
            if score >= threshold
        ]

def tag_event(
    text: str,
    candidate_labels: List[str] = ["sports", "cooking", "gaming", "work", "study", "shopping"]
) -> Dict[str, any]:
    """
    Main workflow: Tag event descriptions
    :param text: Event description text
    :param candidate_labels: Set of candidate tags
    :return: Dictionary with original input and predicted tags
    """
    # Initialize tagger (singleton pattern to avoid reloading model)
    if not hasattr(tag_event, 'tagger'):
        tag_event.tagger = EventTagger()

    # Predict tags
    predicted_tags = tag_event.tagger.predict_tags(text, candidate_labels)

    return {
        "original_text": text,
        "candidate_labels": candidate_labels,
        "predicted_tags": predicted_tags
    }

# Example usage
if __name__ == "__main__":
    # Sample event descriptions
    events = [
        "Need to buy groceries and kitchen supplies at the supermarket tomorrow",
        "Team meeting this afternoon to discuss project requirements and prepare presentation",
        "Playing badminton with friends at the sports center tonight",
        "Researching machine learning algorithms for my thesis project",
        "Cooking a three-course meal for family dinner this weekend"
    ]

    # Execute tagging workflow
    for event in events:
        result = tag_event(event)
        print(f"Input text: {result['original_text']}")
        print(f"Candidate labels: {result['candidate_labels']}")
        print(f"Predicted tags: {result['predicted_tags']}\n{'-'*50}")



### tagger_trainer.py

## Main Function


In [24]:
from google.colab import drive
import pytz

class TodoApp:
    def __init__(self):
        """Initialize todo application"""
        # Mount Google Drive for persistent storage
        drive.mount('/content/drive/')

        # Initialize database
        self.db = DatabaseManager()

        # Initialize NER extractor
        self.ner_extractor = EnhancedNERExtractor()

        # System supported tags
        self.supported_tags = ["sports", "cooking", "gaming", "work", "study", "shopping"]

    def get_formatted_datetime(self):
        """Get current time in New York timezone"""
        ny_tz = pytz.timezone('America/New_York')
        now = datetime.datetime.now(ny_tz)
        return now.strftime("%Y-%m-%d %H:%M:%S %Z%z")

    def print_separator(self):
        """Print timestamped separator"""
        separator = f"=== {self.get_formatted_datetime()} ==="
        print("\n" + separator + "\n")

    def tag_event(self, text):
        """Tag classification workflow"""
        # Initialize tagger on first use
        if not hasattr(self, 'tagger'):
            self.tagger = EventTagger()

        return {
            "original_text": text,
            "candidate_labels": self.supported_tags,
            "predicted_tags": self.tagger.predict_tags(text, self.supported_tags)
        }

    def run(self):
        """Main application loop"""
        while True:
            # Main menu
            print("\n=== Auto-Tagging Todo System ===")
            print("1. Add new todo item")
            print("2. View history")
            print("3. Exit system")
            choice = input("Please select an option: ").strip()
            self.print_separator()

            if choice == "3":
                print("Thank you for using our system. Goodbye!")
                break

            elif choice == "2":
                self.show_history()

            elif choice == "1":
                self.add_todo_item()

            else:
                print("Invalid selection, please try again")

    def add_todo_item(self):
        """Add new todo item workflow"""
        # Step 1: Get todo description
        print("\n--- Step 1: Enter Todo Description ---")
        text = input("Please describe what you plan to do: ").strip()
        self.print_separator()

        # Step 2: Confirm description
        print("\n--- Step 2: Confirm Description ---")
        print(f"Your input: {text}")
        confirm = input("Is this description correct? (y/n): ").lower().strip()
        self.print_separator()

        if confirm != "y":
            print("Please re-enter the description")
            return

        # Step 3: Automatic analysis
        print("\n--- Step 3: Auto Analysis ---")

        # Get current time in New York timezone
        ny_tz = pytz.timezone('America/New_York')
        current_time = datetime.datetime.now(ny_tz)

        # Perform NER extraction
        ner_result = self.ner_extractor.extract_entities(
            text=text,
            current_time=current_time,
            output_format="%Y-%m-%d %H:%M"
        )

        # Perform tag classification
        tagging_result = self.tag_event(text)
        predicted_tags = tagging_result["predicted_tags"]

        # Display NER results
        print("\n--- Named Entity Recognition Results ---")
        print(f"Event Time: {ner_result['event_time'] or 'Not detected'}")
        print(f"Persons: {', '.join(ner_result['persons']) or 'Not detected'}")

        # Display tag classification results
        print("\n--- Tag Classification Results ---")
        print(f"Supported tags: {', '.join(self.supported_tags)}")
        print(f"Predicted tags: {', '.join(predicted_tags) if predicted_tags else 'No tags predicted'}")

        # Step 4: Confirm results
        print("\n--- Step 4: Confirm Results ---")
        confirm_results = input("Are the results correct? (y-correct/n-revise/c-cancel): ").lower().strip()
        self.print_separator()

        if confirm_results == "c":
            print("Operation cancelled")
            return

        # Step 5: Manual tag selection if needed
        if confirm_results == "n":
            print("\n--- Step 5: Manual Tag Selection ---")
            print("Please select tags (enter numbers separated by commas):")
            for i, tag in enumerate(self.supported_tags, 1):
                print(f"{i}. {tag}")

            while True:
                tag_input = input("\nEnter tag numbers: ").strip()
                selected_indices = [s.strip() for s in tag_input.split(",") if s.strip()]

                # Validate input
                valid = True
                selected_tags = []

                for idx in selected_indices:
                    if not idx.isdigit():
                        print(f"Error: '{idx}' is not a valid number")
                        valid = False
                        break

                    num = int(idx)
                    if num < 1 or num > len(self.supported_tags):
                        print(f"Error: Number {num} is out of range (1-{len(self.supported_tags)})")
                        valid = False
                        break

                    selected_tags.append(self.supported_tags[num-1])

                if not valid:
                    continue

                # Confirm selection
                print("\nSelected tags:", ", ".join(selected_tags))
                final_confirm = input("Confirm these tags? (y/n): ").lower().strip()

                if final_confirm == "y":
                    predicted_tags = selected_tags
                    break

        # Step 6: Save results to database
        if confirm_results in ("y", "n"):
            creation_time = self.get_formatted_datetime()

            # Prepare record in required format
            record = {
                "original_text": text,
                "creation_time": creation_time,
                "candidate_labels": self.supported_tags,
                "predicted_tags": predicted_tags,
                "ner_results": {
                    "event_time": ner_result["event_time"],
                    "persons": ner_result["persons"]
                }
            }

            # Save to database
            record_id = self.db.save_record(record)

            # Confirmation message
            print("\n--- Processing Complete ---")
            print(f"Todo item saved successfully! Record ID: {record_id}")
            print(f"Content: {text}")
            print(f"Time: {creation_time}")
            print(f"Tags: {', '.join(predicted_tags)}")
            self.print_separator()

    def show_history(self):
        """Display saved records from database"""
        records = self.db.load_records()

        if not records:
            print("No history records found")
            return

        print(f"\nFound {len(records)} historical records:")
        for record in records:
            print(f"\nID: {record['id']}")
            print(f"Creation Time: {record['creation_time']}")
            print(f"Content: {record['original_text']}")
            print(f"Predicted Tags: {', '.join(record['predicted_tags'])}")

            # Display NER results
            ner = record['ner_results']
            if ner['event_time']:
                print(f"Event Time: {ner['event_time']}")
            if ner['persons']:
                print(f"Persons: {', '.join(ner['persons'])}")

        self.print_separator()

# ======================== Application Entry Point ========================
if __name__ == "__main__":
    # Initialize and run application
    app = TodoApp()
    app.run()


Mounted at /content/drive/
Loaded model: en_core_web_lg

=== Auto-Tagging Todo System ===
1. Add new todo item
2. View history
3. Exit system
Please select an option: 1

=== 2025-08-08 23:06:48 EDT-0400 ===


--- Step 1: Enter Todo Description ---
Please describe what you plan to do: I plan to go play with my younger brother Bob

=== 2025-08-08 23:07:30 EDT-0400 ===


--- Step 2: Confirm Description ---
Your input: I plan to go play with my younger brother Bob
Is this description correct? (y/n): y

=== 2025-08-08 23:07:34 EDT-0400 ===


--- Step 3: Auto Analysis ---


Device set to use cpu



--- Named Entity Recognition Results ---
Event Time: Not detected
Persons: Bob

--- Tag Classification Results ---
Supported tags: sports, cooking, gaming, work, study, shopping
Predicted tags: sports, gaming

--- Step 4: Confirm Results ---
Are the results correct? (y-correct/n-revise/c-cancel): y

=== 2025-08-08 23:08:07 EDT-0400 ===


--- Processing Complete ---
Todo item saved successfully! Record ID: 1
Content: I plan to go play with my younger brother Bob
Time: 2025-08-08 23:08:07 EDT-0400
Tags: sports, gaming

=== 2025-08-08 23:08:07 EDT-0400 ===


=== Auto-Tagging Todo System ===
1. Add new todo item
2. View history
3. Exit system
Please select an option: 2

=== 2025-08-08 23:08:10 EDT-0400 ===


Found 1 historical records:

ID: 1
Creation Time: 2025-08-08 23:08:07 EDT-0400
Content: I plan to go play with my younger brother Bob
Predicted Tags: sports, gaming
Persons: Bob

=== 2025-08-08 23:08:10 EDT-0400 ===


=== Auto-Tagging Todo System ===
1. Add new todo item
2. View histo

In [ ]:
3